In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F


In [4]:
#create a Model Class that inherits nn.Module
class Model(nn.Module) : 
    #input layer (4 features of the flower) --> 
    # Hidden layer1 (#neurons) --> 
    # H2 (n) --> 
    # output (3 classes of iris flowers)

    def __init__(self, in_features=4, h1=8, h2=9, out_features=3) : #the formal parameters define the size of the various layers
        super().__init__() # instantiates our nn.module
        self.fc1 = nn.Linear(in_features, h1) #setting up the layers (fc1 = fully connected 1)
        self.fc2 = nn.Linear(h1, h2) # linear = linear model
        self.out = nn.Linear(h2, out_features)
    
    # forward prop --> this pushes the data through the layers into the output layer
    def forward(self, x) :
        x = F.relu(self.fc1(x)) # rectified linear unit, activation function
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x
    

In [5]:
# picka manual seed for randomization
torch.manual_seed(41)

model = Model()